In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import pandas as pd

Recipe_foodcom = pd.read_csv("drive/My Drive/0_Dataset_Recipes/RAW_recipes.csv", sep = ",")

Recipe_foodcom.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [6]:
# enlever les colones non-essentiels
Recipe_foodcom2 = Recipe_foodcom.drop(['id', 'minutes', 'contributor_id', 'submitted', 'nutrition', 'description'], axis = 1)
Recipe_foodcom2 = Recipe_foodcom2.iloc[:1000,:]

In [4]:
# Definition fonction pour splitter le colone recette et enlever les signes
def pre_process_recipe(recette):
    recette = recette.lower().split("', '")   # spliter
    filtered = []
    for step in recette:
        step = re.sub(r'[\'\,\]\[\-\&\"\/\!\^\#]', ' ', step)     # remplacer les signes par les espaces
        step = re.sub(r'[\s]+', ' ', step)    # remplacer les espaces consécutives en un seul espace
        filtered.append(step)
    #regex2 = r2.findall(text_recette)
    return filtered

In [7]:
# definition fonction pour pré-traiter le texte d'ingrédients

def pre_process_ingrd(ingredient_string):
    # function:  remove special characters, double spaces & punctuations
    ingredient_string.lower()
    ingredient_string = re.sub(r'[\'\,\]\[\-\&\"\/\!\^\#]', ' ', ingredient_string)
    ingredient_string = re.sub(r'[\s]+', ' ', ingredient_string)
    ingredient_string = re.sub(r' and', '', ingredient_string)
    ingredient_string = 'ingredients'+ ingredient_string + '/ingredients' # add word ingredient at the beginning and end of ingredient text
    return ingredient_string

print(Recipe_foodcom2['ingredients'].tolist()[3])
print(pre_process_ingrd(Recipe_foodcom['ingredients'].tolist()[3]))

['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']
ingredients spreadable cheese with garlic herbs new potatoes shallots parsley tarragon olive oil red wine vinegar salt pepper red bell pepper yellow bell pepper /ingredients


In [8]:
def Insert(list, element):
    # function to insert ingredient at the beginning of list of steps, considered as elemnt #0 in the list
    list.insert(0, element)
    return list

In [9]:
# Apply above functions to the "step" column
Recipe_foodcom_p = Recipe_foodcom2
Recipe_foodcom_p['steps'] = Recipe_foodcom_p['steps'].apply(lambda steps : pre_process_recipe(steps))  # split recipe into list of steps
Recipe_foodcom_p['ingredients'] = Recipe_foodcom_p['ingredients'].apply(lambda ingredients : pre_process_ingrd(ingredients)) 
Recipe_foodcom_p['steps']= Recipe_foodcom_p.apply(lambda row: Insert(row['steps'], row['ingredients']), axis = 1)

In [10]:
Recipe_foodcom_p.head(5)
for step in Recipe_foodcom_p['steps'].tolist()[0]:
    print(step)

ingredients winter squash mexican seasoning mixed spice honey butter olive oil salt /ingredients
 make a choice and proceed with recipe
depending on size of squash cut into half or fourths
remove seeds
for spicy squash drizzle olive oil or melted butter over each cut squash piece
season with mexican seasoning mix ii
for sweet squash drizzle melted honey butter grated piloncillo over each cut squash piece
season with sweet mexican spice mix
bake at 350 degrees again depending on size for 40 minutes up to an hour until a fork can easily pierce the skin
be careful not to burn the squash especially if you opt to use sugar or butter
if you feel more comfortable cover the squash with aluminum foil the first half hour give or take of baking
if desired season with salt 


LOAD & SAVE DATAFRAME FILE BEFORE TRAINING FOR REFERENCE

In [17]:
# Saving data into csv file _ for reference later
Recipe_foodcom_p.to_csv(r'drive/My Drive/DataScientest - NLP_Mail/Model1_singleLSTM/Recipe_foodcom_p.csv')

In [18]:
# Loading saved data 
Recipe_foodcom_loaded = pd.read_csv('drive/My Drive/DataScientest - NLP_Mail/Model1_singleLSTM/Recipe_foodcom_p.csv', sep = ",")

# Tokenization

In [25]:
import re
from keras.preprocessing.text import Tokenizer

In [28]:
#Flatten recipe
seq_length = 20
start_recipe = '| ' * seq_length

recipe_list = [' '.join(recipe)for recipe in Recipe_foodcom_p['steps'].tolist()]  # concatenate all "step" column into one single texte
recipe_list = start_recipe.join(map(str,recipe_list))
recipe_list = recipe_list.replace('\n\n\n\n\n', start_recipe)

In [29]:
recipe_list  # single text containing all recipes

'ingredients winter squash mexican seasoning mixed spice honey butter olive oil salt /ingredients  make a choice and proceed with recipe depending on size of squash cut into half or fourths remove seeds for spicy squash drizzle olive oil or melted butter over each cut squash piece season with mexican seasoning mix ii for sweet squash drizzle melted honey butter grated piloncillo over each cut squash piece season with sweet mexican spice mix bake at 350 degrees again depending on size for 40 minutes up to an hour until a fork can easily pierce the skin be careful not to burn the squash especially if you opt to use sugar or butter if you feel more comfortable cover the squash with aluminum foil the first half hour give or take of baking if desired season with salt | | | | | | | | | | | | | | | | | | | | ingredients prepared pizza crust sausage patty eggs milk salt pepper cheese /ingredients  preheat oven to 425 degrees f press dough into the bottom and sides of a 12 inch pizza pan bake f

In [14]:
# TOKENIZATION at word level
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([recipe_list])

total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([recipe_list])[0]
# token_list => serves to feed the tokenised input in next steps

In [ ]:
tokenizer.word_index

# Building the dataset

In [ ]:
import numpy as np
from keras.utils import np_utils

In [ ]:
def generate_sequences(token_list, step):
    X = []
    y = []
    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])                                       # input = sequence of length seq_length
        y.append(token_list[i + seq_length])                                           # output = next word after the sequence
    y = np_utils.to_categorical(y, num_classes = total_words)              # convert each output from one token to one-hot vector
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
 return X, y, num_seq
 
step = 1
seq_length = 20
X, y, num_seq = generate_sequences(token_list, step)
X = np.array(X)
y = np.array(y)

Number of sequences: 131837 



# Building LSTM network


In [ ]:
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop

In [ ]:
seq_length = 20
start_story = '| ' * seq_length

n_units = 256
embedding_size = 100

# Model LSTM  construction 
text_in = Input(shape = (None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(n_units)(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation = 'softmax')(x)  
model = Model(text_in, text_out)

# Compilation
opti = RMSprop(lr = 0.001)
model.compile(loss='categorical_crossentropy', optimizer=opti)

# Training
epochs = 5
batch_size = 16
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/5
8240/8240 [==============================] - 95s 11ms/step - loss: 5.1477
Epoch 2/5
8240/8240 [==============================] - 83s 10ms/step - loss: 5.0656
Epoch 3/5
8240/8240 [==============================] - 85s 10ms/step - loss: 5.5845
Epoch 4/5
8240/8240 [==============================] - 83s 10ms/step - loss: 5.4757
Epoch 5/5
8240/8240 [==============================] - 77s 9ms/step - loss: 5.2982


# Generate text

In [ ]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
 return np.argmax(probs)

def generate_text(seed_text = "recette", next_words = 1, model, max_sequence_len = 5, temp = 1):
 seq_length = 20
 start_story = '|' * seq_length
 output_text = seed_text
 seed_text = start_story + seed_text
 
 for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = token_list[-max_sequence_len:]
      token_list = np.reshape(token_list, (1, max_sequence_len))
      probs = model.predict(token_list, verbose=0)[0]
      y_class = sample_with_temp(probs, temperature = temp)
      output_word = tokenizer.index_word[y_class] if y_class > 0 else ''
      if output_word == "|":
        break
      seed_text += output_word + ' '
      output_text += output_word + ' '
 return output_text

In [ ]:
generate_text("Cut your 3 tomato ", 20, model, 5, 1)

'Cut your 3 tomato minutes remove from heat and set aside with an little spray a large baking dish with the soda vanilla mushrooms '